# Capstone Project: Comparing Boston vs NYC Neighborhoods

## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
pd.options.mode.chained_assignment = None

## Importing and Cleaning the Raw Data to make final Location DataFrame

### Import Zipcodes CSV

In [3]:
US_loc_data = pd.read_csv('zip_code_database.csv', converters={'zip': lambda x: str(x)})

### Cleaning the data and making a table with just New York and Boston Zipcodes and Latitude/Longitude data

In [4]:
MAlocdata = US_loc_data[US_loc_data['state']=='MA']
NYlocdata = US_loc_data[US_loc_data['state']=='NY']

MAlocdata = MAlocdata[(MAlocdata['county']=='Suffolk County')]
MAlocdata.head()

zip      type  decommissioned primary_city acceptable_cities  \
612  02108  STANDARD               0       Boston               NaN   
613  02109  STANDARD               0       Boston               NaN   
614  02110  STANDARD               0       Boston               NaN   
615  02111  STANDARD               0       Boston               NaN   
616  02112    PO BOX               0       Boston               NaN   

    unacceptable_cities state          county          timezone  \
612                 NaN    MA  Suffolk County  America/New_York   
613                 NaN    MA  Suffolk County  America/New_York   
614                 NaN    MA  Suffolk County  America/New_York   
615                 NaN    MA  Suffolk County  America/New_York   
616                 NaN    MA  Suffolk County  America/New_York   

                  area_codes world_region country  latitude  longitude  \
612              617,857,339          NaN      US     42.36     -71.06   
613                      617          NaN      US     42.37     -71.05   
614  508,774,617,781,857,978          NaN      US     42.36     -71.05   
615      617,781,978,339,857          NaN      US     42.35     -71.06   
616                      617          NaN      US     42.35     -71.06   

     irs_estimated_population_2015  
612                           4050  
613                           3980  
614                           3320  
615                           5720  
616                            574

In [5]:
MAlocdata.reset_index(inplace=True)

In [6]:
MAlocdata.drop(columns=['index', 'type', 'decommissioned', 'acceptable_cities', 'unacceptable_cities', 'timezone', 'area_codes', 'country', 'world_region',
                       'irs_estimated_population_2015', 'state'], inplace=True)
MAlocdata.head()

zip primary_city          county  latitude  longitude
0  02108       Boston  Suffolk County     42.36     -71.06
1  02109       Boston  Suffolk County     42.37     -71.05
2  02110       Boston  Suffolk County     42.36     -71.05
3  02111       Boston  Suffolk County     42.35     -71.06
4  02112       Boston  Suffolk County     42.35     -71.06

In [7]:
MAlocdata.rename(columns={'primary_city':'city'}, inplace=True)
MAlocdata.head()

zip    city          county  latitude  longitude
0  02108  Boston  Suffolk County     42.36     -71.06
1  02109  Boston  Suffolk County     42.37     -71.05
2  02110  Boston  Suffolk County     42.36     -71.05
3  02111  Boston  Suffolk County     42.35     -71.06
4  02112  Boston  Suffolk County     42.35     -71.06

In [8]:
NYClocdata = NYlocdata[(NYlocdata['county']=='New York County') | (NYlocdata['county']=='Bronx County') | (NYlocdata['county']=='Kings County') |
                      (NYlocdata['county']=='Queens County') | (NYlocdata['county']=='Richmond County')]
NYClocdata.head()

zip      type  decommissioned primary_city acceptable_cities  \
3699  10001  STANDARD               0     New York               NaN   
3700  10002  STANDARD               0     New York     Knickerbocker   
3701  10003  STANDARD               0     New York               NaN   
3702  10004  STANDARD               0     New York     Bowling Green   
3703  10005  STANDARD               0     New York       Wall Street   

                                    unacceptable_cities state  \
3699  Empire State, G P O, Greeley Square, Macys Fin...    NY   
3700         Manhattan, New York City, Ny, Ny City, Nyc    NY   
3701                                  Cooper, Manhattan    NY   
3702                                                NaN    NY   
3703                                     Manhattan, Nyc    NY   

               county          timezone           area_codes world_region  \
3699  New York County  America/New_York      718,917,347,646          NaN   
3700  New York County  America/New_York                  718          NaN   
3701  New York County  America/New_York  212,347,646,718,917          NaN   
3702  New York County  America/New_York  212,347,646,718,917          NaN   
3703  New York County  America/New_York  347,718,212,646,917          NaN   

     country  latitude  longitude  irs_estimated_population_2015  
3699      US     40.75     -74.00                          19710  
3700      US     40.71     -73.99                          71450  
3701      US     40.73     -73.99                          39470  
3702      US     40.69     -74.02                           4380  
3703      US     40.71     -74.01                           8600

In [9]:
NYClocdata.reset_index(inplace=True)
NYClocdata.drop(columns=['index', 'type', 'decommissioned', 'acceptable_cities', 'unacceptable_cities', 'timezone', 'area_codes', 'country', 'world_region',
                       'irs_estimated_population_2015', 'state'], inplace=True)

In [10]:
NYClocdata.rename(columns={'primary_city':'city'}, inplace=True)

In [11]:
nybosdata = pd.concat([NYClocdata, MAlocdata], ignore_index=True)
nybosdata.rename(columns={'primary_city':'city'}, inplace=True)

### Final Location Data Table

In [12]:
nybosdata

zip                 city           county  latitude  longitude
0    10001             New York  New York County     40.75     -74.00
1    10002             New York  New York County     40.71     -73.99
2    10003             New York  New York County     40.73     -73.99
3    10004             New York  New York County     40.69     -74.02
4    10005             New York  New York County     40.71     -74.01
5    10006             New York  New York County     40.71     -74.01
6    10007             New York  New York County     40.71     -74.01
7    10008             New York  New York County     40.71     -73.99
8    10009             New York  New York County     40.73     -73.98
9    10010             New York  New York County     40.74     -73.98
10   10011             New York  New York County     40.74     -74.00
11   10012             New York  New York County     40.73     -74.00
12   10013             New York  New York County     40.72     -74.00
13   10014             New York  New York County     40.73     -74.01
14   10015             New York  New York County     40.71     -74.00
15   10016             New York  New York County     40.75     -73.98
16   10017             New York  New York County     40.75     -73.97
17   10018             New York  New York County     40.76     -73.99
18   10019             New York  New York County     40.77     -73.99
19   10020             New York  New York County     40.76     -73.98
20   10021             New York  New York County     40.77     -73.96
21   10022             New York  New York County     40.76     -73.97
22   10023             New York  New York County     40.78     -73.98
23   10024             New York  New York County     40.80     -73.97
24   10025             New York  New York County     40.80     -73.97
25   10026             New York  New York County     40.80     -73.95
26   10027             New York  New York County     40.81     -73.95
27   10028             New York  New York County     40.78     -73.95
28   10029             New York  New York County     40.79     -73.94
29   10030             New York  New York County     40.82     -73.94
30   10031             New York  New York County     40.83     -73.95
31   10032             New York  New York County     40.84     -73.94
32   10033             New York  New York County     40.85     -73.93
33   10034             New York  New York County     40.87     -73.92
34   10035             New York  New York County     40.80     -73.93
35   10036             New York  New York County     40.76     -73.99
36   10037             New York  New York County     40.81     -73.94
37   10038             New York  New York County     40.71     -74.00
38   10039             New York  New York County     40.83     -73.94
39   10040             New York  New York County     40.86     -73.93
40   10041             New York  New York County     40.71     -73.99
41   10043             New York  New York County     40.71     -73.99
42   10044             New York  New York County     40.76     -73.95
43   10045             New York  New York County     40.71     -73.99
44   10046             New York  New York County     40.71     -74.01
45   10047             New York  New York County     40.71     -74.01
46   10048             New York  New York County     40.71     -74.01
47   10055             New York  New York County     40.71     -73.99
48   10060             New York  New York County     40.71     -73.99
49   10065             New York  New York County     40.76     -73.96
50   10069             New York  New York County     40.78     -73.99
51   10072             New York  New York County     40.75     -73.99
52   10075             New York  New York County     40.77     -73.95
53   10079             New York  New York County     40.71     -74.00
54   10080             New York  New York County     40.71     -73.99
55   10081             New York  New York County     40.71     -73.99
56   10082       

## Initial maps of NYC and Boston

In [13]:
addressNYC = 'New York City, NY'

geolocator = Nominatim(user_agent='Final_project')
locationNYC = geolocator.geocode(addressNYC)
latNYC = locationNYC.latitude
longNYC = locationNYC.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latNYC, longNYC))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [14]:
addressBOS = 'Boston, MA'

geolocator = Nominatim(user_agent='Final_project')
locationBOS = geolocator.geocode(addressBOS)
latBOS = locationBOS.latitude
longBOS = locationBOS.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latBOS, longBOS))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


### NYC map with zipcode popups

In [15]:
map_newyork = folium.Map(location=[latNYC, longNYC], zoom_start=10)

for lat, lng, zipc, city in zip(NYClocdata['latitude'], NYClocdata['longitude'], NYClocdata['zip'], NYClocdata['city']):
    label = '{}, {}'.format(zipc, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Boston map with zipcode popups

In [16]:
map_boston = folium.Map(location=[latBOS, longBOS], zoom_start=10)

for lat, lng, zipc, city in zip(MAlocdata['latitude'], MAlocdata['longitude'], MAlocdata['zip'], MAlocdata['city']):
    label = '{}, {}'.format(zipc, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

## Bringing in Foursquare data

### Defining Foursquare credentials:

In [17]:
CLIENT_ID = '5PYOPFAT54OJP3NYENDDUIU4RW1DNANTJKABT5TNGBDT2RY2' # your Foursquare ID
CLIENT_SECRET = 'OTARQI2XSFCXBYP2BPKHZCYQAXKQKP1XPG1V2SYAYIISRGNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PYOPFAT54OJP3NYENDDUIU4RW1DNANTJKABT5TNGBDT2RY2
CLIENT_SECRET:OTARQI2XSFCXBYP2BPKHZCYQAXKQKP1XPG1V2SYAYIISRGNK


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
nybosdata.head()

zip      city           county  latitude  longitude
0  10001  New York  New York County     40.75     -74.00
1  10002  New York  New York County     40.71     -73.99
2  10003  New York  New York County     40.73     -73.99
3  10004  New York  New York County     40.69     -74.02
4  10005  New York  New York County     40.71     -74.01

In [20]:
nybos_venues = getNearbyVenues(nybosdata['zip'], nybosdata['latitude'], nybosdata['longitude'])

10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10043
10044
10045
10046
10047
10048
10055
10060
10065
10069
10072
10075
10079
10080
10081
10082
10087
10090
10094
10095
10096
10098
10099
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10128
10129
10130
10131
10132
10133
10138
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10184
10185
10196
10197
10199
10203
10211
10212
10213
10242
10249
10256
10257
10258
10259
10260
10261
10265
10268
10269
10270
10271
10272
10273
10274
10275
10276
10277
10278
10279
10280
10281
10282
10285
10286
10292
10301
10302
1030

In [75]:
nybosdata.shape

(394, 5)

In [76]:
nybos_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0        10001                  40.75                   -74.0   
1        10001                  40.75                   -74.0   
2        10001                  40.75                   -74.0   
3        10001                  40.75                   -74.0   
4        10001                  40.75                   -74.0   

                      Venue  Venue Latitude  Venue Longitude  \
0            Bluestone Lane       40.752254       -73.998824   
1                   Porteño       40.750443       -74.002407   
2         Jun-Men Ramen Bar       40.747956       -74.000301   
3  26th Street Viewing Spur       40.749825       -74.003352   
4             Hudson Market       40.750678       -74.002087   

           Venue Category  
0             Coffee Shop  
1  Argentinian Restaurant  
2        Ramen Restaurant  
3          Scenic Lookout  
4           Grocery Store

In [77]:
nybos_venues.groupby('Neighborhood').count().head()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                         
02108                           100                     100    100   
02109                            23                      23     23   
02110                           100                     100    100   
02111                           100                     100    100   
02112                           100                     100    100   

              Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                   
02108                    100              100             100  
02109                     23               23              23  
02110                    100              100             100  
02111                    100              100             100  
02112                    100              100             100

## Analyzing the data

### One-hot encoding:

In [78]:
nybos_onehot = pd.get_dummies(nybos_venues[['Venue Category']], prefix="", prefix_sep="")
nybos_onehot['Neighborhood'] = nybos_venues['Neighborhood']
fixed_columns = [nybos_onehot.columns[-1]] + list(nybos_onehot.columns[:-1])
nybos_onehot = nybos_onehot[fixed_columns]

nybos_onehot.head()

Zoo Exhibit  ATM  Accessories Store  Adult Boutique  Afghan Restaurant  \
0            0    0                  0               0                  0   
1            0    0                  0               0                  0   
2            0    0                  0               0                  0   
3            0    0                  0               0                  0   
4            0    0                  0               0                  0   

   African Restaurant  Airport  Airport Lounge  Airport Service  \
0                   0        0               0                0   
1                   0        0               0                0   
2                   0        0               0                0   
3                   0        0               0                0   
4                   0        0               0                0   

   Airport Terminal  Airport Tram  American Restaurant  Animal Shelter  \
0                 0             0                    0               0   
1                 0             0                    0               0   
2                 0             0                    0               0   
3                 0             0                    0               0   
4                 0             0                    0               0   

   Antique Shop  Aquarium  Arcade  Arepa Restaurant  Argentinian Restaurant  \
0             0         0       0                 0                       0   
1             0         0       0                 0                       1   
2             0         0       0                 0                       0   
3             0         0       0                 0                       0   
4             0         0       0                 0                       0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Athletics & Sports  Auditorium  Australian Restaurant  Auto Garage  \
0                   0           0                      0            0   
1                   0           0                      0            0   
2                   0           0                      0            0   
3                   0           0                      0            0   
4                   0           0                      0            0   

   Automotive Shop  BBQ Joint  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0                0          0           0       0     0    0               0   
1                0          0           0       0     0    0               0   
2                0          0           0       0     0    0               0   
3                0          0           0       0     0    0               0   
4                0          0           0       0     0    0               0   

   Baseball Stadium  Basketball Court  Basketball Stadium  Beach  Beach Bar  \
0                 0                 0                   0      0          0   
1                 0                 0                   0      0          0   
2                 0                 0                   0      0          0   
3                 0                 0                   0      0          0   
4                 0                 0                   0      0          0   

   Bed & Breakfast  Beer Bar  Beer Garden  Beer Store  Belgian Restaurant  \
0                0         0            0           0                   0   
1                0         0            0           0                   0   
2                0         0            0           0                   0   
3                0         0            0           0                   0   
4                0    

In [79]:
nybos_onehot.shape

(16969, 441)

In [80]:
nybos_grouped = nybos_onehot.groupby('Neighborhood').mean().reset_index()
nybos_grouped.head()

Neighborhood  Zoo Exhibit  ATM  Accessories Store  Adult Boutique  \
0        02108          0.0  0.0                0.0             0.0   
1        02109          0.0  0.0                0.0             0.0   
2        02110          0.0  0.0                0.0             0.0   
3        02111          0.0  0.0                0.0             0.0   
4        02112          0.0  0.0                0.0             0.0   

   Afghan Restaurant  African Restaurant  Airport  Airport Lounge  \
0                0.0                 0.0      0.0             0.0   
1                0.0                 0.0      0.0             0.0   
2                0.0                 0.0      0.0             0.0   
3                0.0                 0.0      0.0             0.0   
4                0.0                 0.0      0.0             0.0   

   Airport Service  Airport Terminal  Airport Tram  American Restaurant  \
0              0.0               0.0           0.0                 0.05   
1              0.0               0.0           0.0                 0.00   
2              0.0               0.0           0.0                 0.03   
3              0.0               0.0           0.0                 0.01   
4              0.0               0.0           0.0                 0.01   

   Animal Shelter  Antique Shop  Aquarium  Arcade  Arepa Restaurant  \
0             0.0           0.0      0.00     0.0               0.0   
1             0.0           0.0      0.00     0.0               0.0   
2             0.0           0.0      0.04     0.0               0.0   
3             0.0           0.0      0.00     0.0               0.0   
4             0.0           0.0      0.00     0.0               0.0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                     0.0          0.0         0.0                  0.0   
1                     0.0          0.0         0.0                  0.0   
2                     0.0          0.0         0.0                  0.0   
3                     0.0          0.0         0.0                  0.0   
4                     0.0          0.0         0.0                  0.0   

   Asian Restaurant  Athletics & Sports  Auditorium  Australian Restaurant  \
0              0.00            0.000000         0.0                    0.0   
1              0.00            0.043478         0.0                    0.0   
2              0.01            0.000000         0.0                    0.0   
3              0.08            0.000000         0.0                    0.0   
4              0.08            0.000000         0.0                    0.0   

   Auto Garage  Automotive Shop  BBQ Joint  Bagel Shop  Bakery  Bank   Bar  \
0          0.0              0.0        0.0        0.01    0.04   0.0  0.01   
1          0.0              0.0        0.0        0.00    0.00   0.0  0.00   
2          0.0              0.0        0.0        0.00    0.02   0.0  0.04   
3          0.0              0.0        0.0        0.00    0.06   0.0  0.00   
4          0.0              0.0        0.0        0.00    0.06   0.0  0.00   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0             0.0               0.0               0.0                 0.0   
1             0.0               0.0               0.0                 0.0   
2             0.0               0.0               0.0                 0.0   
3             0.0               0.0               0.0                 0.0   
4             0.0               0.0               0.0                 0.0   

   Beach  Beach Bar  Bed & Breakfast  Beer Bar  Beer Garden  Beer Store  \
0    0.0        0.0              0.0      0.00         0.01         0.0   
1    0.0        0.0              0.0      0.00         0.00         0.0   
2    0.0        0.0              0.0      0.01         0.01         0.0   
3    0.0        0.0              0.0      0.00         0.00         0.0   
4    0.0        0.0              0.0      0.00         

Confirm shape of nybos_grouped:

In [81]:
nybos_grouped.shape

(394, 441)

Confirm that no rows were lost during onehot encoding and subsequent grouping:

In [82]:
nybos_grouped.shape[0] == nybos_venues.groupby('Neighborhood').count().shape[0]

True

### Dataframe with top 10 venues per zip:

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zip_venues_sorted = pd.DataFrame(columns=columns)
zip_venues_sorted['Neighborhood'] = nybos_grouped['Neighborhood']

for ind in np.arange(nybos_grouped.shape[0]):
    zip_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nybos_grouped.iloc[ind, :], num_top_venues)

zip_venues_sorted.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0        02108         Historic Site    Seafood Restaurant   
1        02109       Harbor / Marina        History Museum   
2        02110         Boat or Ferry    Seafood Restaurant   
3        02111      Asian Restaurant    Chinese Restaurant   
4        02112      Asian Restaurant    Chinese Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Coffee Shop        Sandwich Place   American Restaurant   
1         Boat or Ferry                   Spa                  Park   
2       Harbor / Marina    Italian Restaurant           Coffee Shop   
3                Bakery           Coffee Shop      Sushi Restaurant   
4                Bakery           Coffee Shop      Sushi Restaurant   

  6th Most Common Venue       7th Most Common Venue 8th Most Common Venue  \
0                 Hotel          Italian Restaurant                Bakery   
1                   Gym  Tourist Information Center    Athletics & Sports   
2                  Park                    Aquarium                   Bar   
3               Theater              Sandwich Place           Pizza Place   
4               Theater              Sandwich Place           Pizza Place   

  9th Most Common Venue   10th Most Common Venue  
0            Restaurant  New American Restaurant  
1                Tunnel                     Café  
2                 Hotel           Sandwich Place  
3                   Gym    Performing Arts Venue  
4                   Gym    Performing Arts Venue

## Cluster the Zips:

In [85]:
kclusters = 10
nybos_grouped_clustering = nybos_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nybos_grouped_clustering)

kmeans.labels_

array([0, 0, 0, 8, 8, 0, 0, 0, 0, 8, 0, 0, 4, 0, 0, 8, 4, 0, 4, 0, 0, 0,
       0, 8, 4, 8, 8, 8, 0, 4, 4, 0, 0, 0, 8, 0, 8, 0, 8, 8, 8, 0, 0, 8,
       8, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 2, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       8, 8, 8, 8, 8, 0, 0, 0, 0, 8, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0,
       0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0,
       0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 3, 2,
       2, 2, 5, 8, 5, 5, 5, 3, 5, 8, 4, 5, 0, 9, 3, 0, 0, 4, 4, 4, 8, 0,
       4, 4, 8, 8, 4, 4, 4, 6, 3, 4, 3, 4, 4, 7, 3, 4, 0, 4, 4, 8, 3, 4,
       0, 8, 4, 8, 8, 8, 0, 0, 8, 0, 3, 3, 4, 0, 0, 3, 3, 0, 4, 0, 3, 8,
       0, 0, 0, 0, 4, 8, 8, 0, 0, 8, 0, 8, 4, 8, 4,

In [86]:
nybosdata.rename(columns={'zip':'Neighborhood'}, inplace=True)

nybos_merged = pd.merge(nybosdata,zip_venues_sorted, how='outer', on='Neighborhood', indicator=True)

nybos_merged = nybos_merged[nybos_merged['_merge']=='both']

drop_list = list(zip_venues_sorted.columns)
drop_list.pop(0)
drop_list.append('_merge')
drop_list

['1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue',
 '_merge']

In [87]:
nybos_merged = nybos_merged.drop(columns=drop_list)

In [88]:
nybos_merged['Cluster'] = kmeans.labels_

In [89]:
nybos_merged.head()

Neighborhood      city           county  latitude  longitude  Cluster
0        10001  New York  New York County     40.75     -74.00        0
1        10002  New York  New York County     40.71     -73.99        0
2        10003  New York  New York County     40.73     -73.99        0
3        10004  New York  New York County     40.69     -74.02        8
4        10005  New York  New York County     40.71     -74.01        8

In [90]:
nybos_merged = pd.merge(nybos_merged,zip_venues_sorted, how='outer', on='Neighborhood', indicator=True)
nybos_merged.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
0        10001  New York  New York County     40.75     -74.00        0   
1        10002  New York  New York County     40.71     -73.99        0   
2        10003  New York  New York County     40.73     -73.99        0   
3        10004  New York  New York County     40.69     -74.02        8   
4        10005  New York  New York County     40.71     -74.01        8   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0           Art Gallery           Coffee Shop         Deli / Bodega   
1                  Café      Malay Restaurant           Art Gallery   
2   Japanese Restaurant         Grocery Store    Italian Restaurant   
3            Food Truck    Seafood Restaurant                  Café   
4           Coffee Shop                 Hotel         Memorial Site   

  4th Most Common Venue 5th Most Common Venue     6th Most Common Venue  \
0    Chinese Restaurant           Pizza Place              Cocktail Bar   
1     French Restaurant  Other Great Outdoors                     Motel   
2           Coffee Shop          Dessert Shop            Ice Cream Shop   
3            Food Stand        Ice Cream Shop  Bike Rental / Bike Share   
4           Pizza Place        Sandwich Place                       Bar   

  7th Most Common Venue 8th Most Common Venue    9th Most Common Venue  \
0      Tapas Restaurant                Lounge  Health & Beauty Service   
1  Cantonese Restaurant    Mexican Restaurant             Skating Rink   
2                   Gym           Yoga Studio       Chinese Restaurant   
3           Beer Garden  Gym / Fitness Center                      Bar   
4                  Park                  Café               Food Truck   

  10th Most Common Venue _merge  
0                Theater   both  
1             Skate Park   both  
2                    Spa   both  
3         Baseball Field   both  
4    American Restaurant   both

### Visualizing the Clusters:

In [91]:
map_clusters = folium.Map(location=[39.828273, -98.579456], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nybos_merged['latitude'], nybos_merged['longitude'], nybos_merged['Neighborhood'], nybos_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Splitting the nyla_merged dataframe by cluster

### Cluster 1:

In [92]:
nybos_cluster1 = nybos_merged[nybos_merged['Cluster']==1]
nybos_cluster1.head()

Neighborhood    city          county  latitude  longitude  Cluster  \
379        02211  Boston  Suffolk County     42.35     -71.06        1   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
379      Asian Restaurant    Chinese Restaurant                Bakery   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
379           Coffee Shop      Sushi Restaurant               Theater   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
379        Sandwich Place           Pizza Place                   Gym   

    10th Most Common Venue _merge  
379  Performing Arts Venue   both

In [93]:
nyc_cluster1 = nybos_cluster1[(nybos_cluster1['county']=='New York County') | (nybos_cluster1['county']=='Bronx County') | (nybos_cluster1['county']=='Kings County') |
                      (nybos_cluster1['county']=='Queens County') | (nybos_cluster1['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 1.'.format(nyc_cluster1.shape[0]))

There are 0 New York City zipcodes in cluster 1.


In [96]:
bos_cluster1 = nybos_cluster1[nybos_cluster1['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 1.'.format(bos_cluster1.shape[0]))

There are 1 Boston zipcodes in cluster 1.


### Cluster 2:

In [97]:
nybos_cluster2 = nybos_merged[nybos_merged['Cluster']==2]
nybos_cluster2.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
59         10094  New York  New York County     40.71     -74.00        2   
65         10102  New York  New York County     40.71     -73.99        2   
98         10150  New York  New York County     40.71     -73.99        2   
99         10151  New York  New York County     40.71     -73.99        2   
101        10153  New York  New York County     40.76     -73.97        2   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
59                   Café        Ice Cream Shop    Chinese Restaurant   
65                   Café      Malay Restaurant           Art Gallery   
98                   Café      Malay Restaurant           Art Gallery   
99                   Café      Malay Restaurant           Art Gallery   
101                   Spa   American Restaurant           Coffee Shop   

    4th Most Common Venue 5th Most Common Venue    6th Most Common Venue  \
59     Italian Restaurant    Dim Sum Restaurant                      Bar   
65      French Restaurant  Other Great Outdoors                    Motel   
98      French Restaurant  Other Great Outdoors                    Motel   
99      French Restaurant  Other Great Outdoors                    Motel   
101                 Hotel              Boutique  New American Restaurant   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
59                 Bakery         Grocery Store           Coffee Shop   
65   Cantonese Restaurant    Mexican Restaurant          Skating Rink   
98   Cantonese Restaurant    Mexican Restaurant          Skating Rink   
99   Cantonese Restaurant    Mexican Restaurant          Skating Rink   
101    Italian Restaurant    Chinese Restaurant    Salon / Barbershop   

    10th Most Common Venue _merge  
59      Seafood Restaurant   both  
65              Skate Park   both  
98              Skate Park   both  
99              Skate Park   both  
101                 Bakery   both

In [98]:
nyc_cluster2 = nybos_cluster2[(nybos_cluster2['county']=='New York County') | (nybos_cluster2['county']=='Bronx County') | (nybos_cluster2['county']=='Kings County') |
                      (nybos_cluster2['county']=='Queens County') | (nybos_cluster2['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 2.'.format(nyc_cluster2.shape[0]))

There are 81 New York City zipcodes in cluster 2.


In [99]:
bos_cluster2 = nybos_cluster2[nybos_cluster2['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 2.'.format(bos_cluster2.shape[0]))

There are 0 Boston zipcodes in cluster 2.


### Cluster 3:

In [100]:
nybos_cluster3 = nybos_merged[nybos_merged['Cluster']==3]
nybos_cluster3.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
86         10123  New York  New York County     40.71     -73.99        3   
218        11204  Brooklyn     Kings County     40.62     -73.98        3   
227        11213  Brooklyn     Kings County     40.67     -73.94        3   
234        11220  Brooklyn     Kings County     40.64     -74.02        3   
250        11237  Brooklyn     Kings County     40.70     -73.92        3   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
86                   Café      Malay Restaurant           Art Gallery   
218              Pharmacy         Grocery Store           Pizza Place   
227           Pizza Place         Grocery Store                  Café   
234           Pizza Place    Mexican Restaurant         Grocery Store   
250           Coffee Shop    Mexican Restaurant                   Bar   

         4th Most Common Venue 5th Most Common Venue  \
86           French Restaurant  Other Great Outdoors   
218  Middle Eastern Restaurant                   Zoo   
227              Deli / Bodega   Fried Chicken Joint   
234             Ice Cream Shop                Bakery   
250                Pizza Place         Deli / Bodega   

         6th Most Common Venue 7th Most Common Venue  8th Most Common Venue  \
86                       Motel  Cantonese Restaurant     Mexican Restaurant   
218             Farmers Market    English Restaurant  Entertainment Service   
227                Candy Store        Clothing Store             Bagel Shop   
234        American Restaurant                School         Sandwich Place   
250  Latin American Restaurant              Dive Bar     Italian Restaurant   

    9th Most Common Venue  10th Most Common Venue _merge  
86           Skating Rink              Skate Park   both  
218  Ethiopian Restaurant             Event Space   both  
227           Coffee Shop          Sandwich Place   both  
234           Coffee Shop            Dessert Shop   both  
250                Bakery  Thrift / Vintage Store   both

In [101]:
nyc_cluster3 = nybos_cluster3[(nybos_cluster3['county']=='New York County') | (nybos_cluster3['county']=='Bronx County') | (nybos_cluster3['county']=='Kings County') |
                      (nybos_cluster3['county']=='Queens County') | (nybos_cluster3['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 3.'.format(nyc_cluster3.shape[0]))

There are 23 New York City zipcodes in cluster 3.


In [102]:
bos_cluster3 = nybos_cluster3[nybos_cluster3['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 3.'.format(bos_cluster3.shape[0]))

There are 6 Boston zipcodes in cluster 3.


### Cluster 4

In [103]:
nybos_cluster4 = nybos_merged[nybos_merged['Cluster']==4]
nybos_cluster4.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
12        10013  New York  New York County     40.72     -74.00        4   
16        10017  New York  New York County     40.75     -73.97        4   
18        10019  New York  New York County     40.77     -73.99        4   
24        10025  New York  New York County     40.80     -73.97        4   
29        10030  New York  New York County     40.82     -73.94        4   

              1st Most Common Venue 2nd Most Common Venue  \
12                   Clothing Store         Women's Store   
16               Italian Restaurant   Japanese Restaurant   
18                             Park  Gym / Fitness Center   
24                      Pizza Place    Chinese Restaurant   
29  Southern / Soul Food Restaurant           Bus Station   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
12                  Café           Yoga Studio           Men's Store   
16           Coffee Shop      Sushi Restaurant            Steakhouse   
18               Theater           Coffee Shop      Sculpture Garden   
24    Mexican Restaurant           Coffee Shop     Indian Restaurant   
29             BBQ Joint   Rental Car Location        Baseball Field   

   6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
12                Bakery  Furniture / Home Store           Art Gallery   
16                  Park        Asian Restaurant         Deli / Bodega   
18             Hotel Bar                Wine Bar                 Hotel   
24    Italian Restaurant              Bagel Shop         Grocery Store   
29                Lounge            Tennis Court      Tapas Restaurant   

        9th Most Common Venue 10th Most Common Venue _merge  
12   Mediterranean Restaurant         Ice Cream Shop   both  
16                     Bakery     Seafood Restaurant   both  
18                  Gastropub            Pizza Place   both  
24               Noodle House           Liquor Store   both  
29  Middle Eastern Restaurant    Fried Chicken Joint   both

In [104]:
nyc_cluster4 = nybos_cluster4[(nybos_cluster4['county']=='New York County') | (nybos_cluster4['county']=='Bronx County') | (nybos_cluster4['county']=='Kings County') |
                      (nybos_cluster4['county']=='Queens County') | (nybos_cluster4['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 4.'.format(nyc_cluster4.shape[0]))

There are 33 New York City zipcodes in cluster 4.


In [105]:
bos_cluster4 = nybos_cluster4[nybos_cluster4['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 4.'.format(bos_cluster4.shape[0]))

There are 15 Boston zipcodes in cluster 4.


### Cluster 5:

In [106]:
nybos_cluster5 = nybos_merged[nybos_merged['Cluster']==5]
nybos_cluster5.head()

Neighborhood      city        county  latitude  longitude  Cluster  \
222        11208  Brooklyn  Kings County     40.67     -73.87        5   
224        11210  Brooklyn  Kings County     40.63     -73.95        5   
225        11211  Brooklyn  Kings County     40.71     -73.95        5   
226        11212  Brooklyn  Kings County     40.66     -73.91        5   
228        11214  Brooklyn  Kings County     40.60     -74.00        5   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
222         Deli / Bodega        Sandwich Place        Discount Store   
224     Mobile Phone Shop  Caribbean Restaurant              Pharmacy   
225                   Bar           Coffee Shop           Pizza Place   
226            Restaurant   Fried Chicken Joint      Airport Terminal   
228   Rental Car Location        Ice Cream Shop            Hookah Bar   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
222            Food Truck           Supermarket           Pizza Place   
224    Chinese Restaurant         Deli / Bodega            Restaurant   
225    Italian Restaurant    Mexican Restaurant                Bakery   
226                  Park        Discount Store           Pizza Place   
228                  Park   Shanghai Restaurant            Donut Shop   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
222              Bus Stop           Yoga Studio                  Park   
224        Breakfast Spot           Coffee Shop   Sporting Goods Shop   
225   Japanese Restaurant         Deli / Bodega    Chinese Restaurant   
226  Caribbean Restaurant              Bus Line    Chinese Restaurant   
228           Pizza Place       Supplement Shop             Surf Spot   

      10th Most Common Venue _merge  
222       Chinese Restaurant   both  
224           Shipping Store   both  
225  New American Restaurant   both  
226            Moving Target   both  
228               Restaurant   both

In [107]:
nyc_cluster5 = nybos_cluster5[(nybos_cluster5['county']=='New York County') | (nybos_cluster5['county']=='Bronx County') | (nybos_cluster5['county']=='Kings County') |
                      (nybos_cluster5['county']=='Queens County') | (nybos_cluster5['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 5.'.format(nyc_cluster1.shape[0]))

There are 0 New York City zipcodes in cluster 5.


In [108]:
bos_cluster5 = nybos_cluster5[nybos_cluster5['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 5.'.format(bos_cluster5.shape[0]))

There are 2 Boston zipcodes in cluster 5.


### Cluster 6:

In [109]:
nybos_cluster6 = nybos_merged[nybos_merged['Cluster']==6]
nybos_cluster6.head()

Neighborhood      city        county  latitude  longitude  Cluster  \
249        11236  Brooklyn  Kings County     40.64      -73.9        6   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
249  Caribbean Restaurant        Cosmetics Shop            Taco Place   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
249            Bagel Shop                   Gym           Pizza Place   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
249  Fast Food Restaurant     Mobile Phone Shop     Martial Arts Dojo   

    10th Most Common Venue _merge  
249                  Diner   both

In [110]:
nyc_cluster6 = nybos_cluster6[(nybos_cluster6['county']=='New York County') | (nybos_cluster6['county']=='Bronx County') | (nybos_cluster6['county']=='Kings County') |
                      (nybos_cluster6['county']=='Queens County') | (nybos_cluster6['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 6.'.format(nyc_cluster6.shape[0]))

There are 1 New York City zipcodes in cluster 6.


In [111]:
bos_cluster6 = nybos_cluster6[nybos_cluster6['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 6.'.format(bos_cluster6.shape[0]))

There are 0 Boston zipcodes in cluster 6.


### Cluster 7:

In [112]:
nybos_cluster7 = nybos_merged[nybos_merged['Cluster']==7]
nybos_cluster7.head()

Neighborhood      city        county  latitude  longitude  Cluster  \
255        11242  Brooklyn  Kings County     40.64     -73.94        7   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
255           Supermarket            Playground   Rental Car Location   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
255     Food & Drink Shop                  Food         Moving Target   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
255  Fast Food Restaurant    Mexican Restaurant                  Park   

    10th Most Common Venue _merge  
255   Caribbean Restaurant   both

In [113]:
nyc_cluster7 = nybos_cluster7[(nybos_cluster7['county']=='New York County') | (nybos_cluster7['county']=='Bronx County') | (nybos_cluster7['county']=='Kings County') |
                      (nybos_cluster7['county']=='Queens County') | (nybos_cluster7['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 7.'.format(nyc_cluster7.shape[0]))

There are 1 New York City zipcodes in cluster 7.


In [114]:
bos_cluster7 = nybos_cluster7[nybos_cluster7['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 7.'.format(bos_cluster7.shape[0]))

There are 0 Boston zipcodes in cluster 7.


### Cluster 8:

In [115]:
nybos_cluster8 = nybos_merged[nybos_merged['Cluster']==8]
nybos_cluster8.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
3         10004  New York  New York County     40.69     -74.02        8   
4         10005  New York  New York County     40.71     -74.01        8   
9         10010  New York  New York County     40.74     -73.98        8   
15        10016  New York  New York County     40.75     -73.98        8   
23        10024  New York  New York County     40.80     -73.97        8   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
3             Food Truck    Seafood Restaurant                  Café   
4            Coffee Shop                 Hotel         Memorial Site   
9      Indian Restaurant   American Restaurant        Ice Cream Shop   
15                 Hotel           Coffee Shop        Sandwich Place   
23           Pizza Place    Chinese Restaurant    Mexican Restaurant   

   4th Most Common Venue 5th Most Common Venue     6th Most Common Venue  \
3             Food Stand        Ice Cream Shop  Bike Rental / Bike Share   
4            Pizza Place        Sandwich Place                       Bar   
9     Italian Restaurant                   Bar               Coffee Shop   
15   Japanese Restaurant    Italian Restaurant      Gym / Fitness Center   
23           Coffee Shop     Indian Restaurant        Italian Restaurant   

   7th Most Common Venue   8th Most Common Venue 9th Most Common Venue  \
3            Beer Garden    Gym / Fitness Center                   Bar   
4                   Park                    Café            Food Truck   
9          Grocery Store  Thrift / Vintage Store                 Hotel   
15    Chinese Restaurant     American Restaurant      Cuban Restaurant   
23            Bagel Shop           Grocery Store          Noodle House   

   10th Most Common Venue _merge  
3          Baseball Field   both  
4     American Restaurant   both  
9            Cocktail Bar   both  
15             Steakhouse   both  
23           Liquor Store   both

In [116]:
nyc_cluster8 = nybos_cluster8[(nybos_cluster8['county']=='New York County') | (nybos_cluster8['county']=='Bronx County') | (nybos_cluster8['county']=='Kings County') |
                      (nybos_cluster8['county']=='Queens County') | (nybos_cluster8['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 8.'.format(nyc_cluster8.shape[0]))

There are 52 New York City zipcodes in cluster 8.


In [117]:
bos_cluster8 = nybos_cluster8[nybos_cluster8['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 8.'.format(bos_cluster8.shape[0]))

There are 20 Boston zipcodes in cluster 8.


### Cluster 9:

In [118]:
nybos_cluster9 = nybos_merged[nybos_merged['Cluster']==9]
nybos_cluster9.head()

Neighborhood         city          county  latitude  longitude  Cluster  \
233        11219     Brooklyn    Kings County     40.63     -74.00        9   
357        02129  Charlestown  Suffolk County     42.38     -71.06        9   
378        02210       Boston  Suffolk County     42.35     -71.04        9   
381        02215       Boston  Suffolk County     42.35     -71.10        9   
384        02222       Boston  Suffolk County     42.35     -71.06        9   

    1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
233    Chinese Restaurant           Supermarket    Italian Restaurant   
357           Pizza Place          Liquor Store         Boat or Ferry   
378    Seafood Restaurant                  Café            Donut Shop   
381            Sports Bar           Coffee Shop   American Restaurant   
384      Asian Restaurant    Chinese Restaurant                Bakery   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
233           Bus Station                   Bar                  Bank   
357   Monument / Landmark             Gastropub         Historic Site   
378            Steakhouse   American Restaurant            Art Museum   
381                 Hotel                Lounge         Hot Dog Joint   
384           Coffee Shop      Sushi Restaurant               Theater   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
233           Event Space        Hardware Store  Cantonese Restaurant   
357        History Museum        Discount Store                  Café   
378       Harbor / Marina                   Gym           Music Venue   
381  Fast Food Restaurant            Donut Shop           Pizza Place   
384        Sandwich Place           Pizza Place                   Gym   

    10th Most Common Venue _merge  
233     Seafood Restaurant   both  
357            Dry Cleaner   both  
378            Coffee Shop   both  
381         Baseball Field   both  
384  Performing Arts Venue   both

In [119]:
nyc_cluster9 = nybos_cluster9[(nybos_cluster9['county']=='New York County') | (nybos_cluster9['county']=='Bronx County') | (nybos_cluster9['county']=='Kings County') |
                      (nybos_cluster9['county']=='Queens County') | (nybos_cluster9['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 9.'.format(nyc_cluster9.shape[0]))

There are 1 New York City zipcodes in cluster 9.


In [121]:
bos_cluster9 = nybos_cluster9[nybos_cluster9['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 9.'.format(bos_cluster9.shape[0]))

There are 5 Boston zipcodes in cluster 9.


### Cluster 0:

In [122]:
nybos_cluster0 = nybos_merged[nybos_merged['Cluster']==0]
nybos_cluster0.head()

Neighborhood      city           county  latitude  longitude  Cluster  \
0        10001  New York  New York County     40.75     -74.00        0   
1        10002  New York  New York County     40.71     -73.99        0   
2        10003  New York  New York County     40.73     -73.99        0   
5        10006  New York  New York County     40.71     -74.01        0   
6        10007  New York  New York County     40.71     -74.01        0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0           Art Gallery           Coffee Shop         Deli / Bodega   
1                  Café      Malay Restaurant           Art Gallery   
2   Japanese Restaurant         Grocery Store    Italian Restaurant   
5           Coffee Shop                 Hotel         Memorial Site   
6           Coffee Shop                 Hotel         Memorial Site   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0    Chinese Restaurant           Pizza Place          Cocktail Bar   
1     French Restaurant  Other Great Outdoors                 Motel   
2           Coffee Shop          Dessert Shop        Ice Cream Shop   
5           Pizza Place        Sandwich Place                   Bar   
6           Pizza Place        Sandwich Place                   Bar   

  7th Most Common Venue 8th Most Common Venue    9th Most Common Venue  \
0      Tapas Restaurant                Lounge  Health & Beauty Service   
1  Cantonese Restaurant    Mexican Restaurant             Skating Rink   
2                   Gym           Yoga Studio       Chinese Restaurant   
5                  Park                  Café               Food Truck   
6                  Park                  Café               Food Truck   

  10th Most Common Venue _merge  
0                Theater   both  
1             Skate Park   both  
2                    Spa   both  
5    American Restaurant   both  
6    American Restaurant   both

In [123]:
nyc_cluster0 = nybos_cluster0[(nybos_cluster0['county']=='New York County') | (nybos_cluster0['county']=='Bronx County') | (nybos_cluster0['county']=='Kings County') |
                      (nybos_cluster0['county']=='Queens County') | (nybos_cluster0['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 0.'.format(nyc_cluster0.shape[0]))

There are 138 New York City zipcodes in cluster 0.


In [124]:
bos_cluster0 = nybos_cluster0[nybos_cluster0['county']=='Suffolk County']
print('There are {} Boston zipcodes in cluster 0.'.format(bos_cluster0.shape[0]))

There are 9 Boston zipcodes in cluster 0.
